###Replicate commonsense reasoning tasks

In [11]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

import torch
torch.cuda.empty_cache()

In [ ]:
!pwd
!pip list

/content/DoRA/commonsense_reasoning
Package                                  Version
---------------------------------------- ------------------
absl-py                                  1.4.0
accelerate                               0.26.1
access                                   1.1.10.post3
affine                                   2.4.0
aiofiles                                 23.2.1
aiohappyeyeballs                         2.6.1
aiohttp                                  3.13.3
aiosignal                                1.4.0
aiosqlite                                0.22.1
alabaster                                1.0.0
albucore                                 0.0.24
albumentations                           2.0.8
ale-py                                   0.11.2
alembic                                  1.17.2
altair                                   5.5.0
annotated-doc                            0.0.4
annotated-types                          0.7.0
antlr4-python3-runtime                   4

In [ ]:
!nvidia-smi

Thu Jan 15 23:06:55 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!git clone https://github.com/NVlabs/DoRA.git
%cd /content/DoRA/commonsense_reasoning


!pip uninstall peft bitsandbytes -y

fatal: destination path 'DoRA' already exists and is not an empty directory.
/content/DoRA/commonsense_reasoning
Found existing installation: bitsandbytes 0.49.1
Uninstalling bitsandbytes-0.49.1:
  Successfully uninstalled bitsandbytes-0.49.1


In [ ]:
!pip install -r requirements.txt

# for cuda and memory cache clear
!pip install bitsandbytes --upgrade
!pip install accelerate==0.26.1

  Cloning https://github.com/huggingface/transformers.git (to revision v4.36.0) to /tmp/pip-req-build-1cx0yejt
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-1cx0yejt
  Running command git checkout -q 6da8b7cbc9ded50361b0428a5725d9b9410ca169
  Resolved https://github.com/huggingface/transformers.git to commit 6da8b7cbc9ded50361b0428a5725d9b9410ca169
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached bitsandbytes-0.41.3.post2-py3-none-any.whl.metadata (9.8 kB)
Using cached bitsandbytes-0.41.3.post2-py3-none-any.whl (92.6 MB)
  Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.49.1-py3-none-manylinux_2_24_x86_64.whl (59.1 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.41.3.post2
    Uninstalling bitsandbytes-0.41.

###170k finetuning dataset

In [ ]:
!rm commonsense_170k.json
!wget https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/ft-training_set/commonsense_170k.json -O commonsense_170k.json

if os.path.exists("commonsense_170k.json"):
    print("Dataset downloaded successfully!")
else:
    print("Error: Dataset download failed.")

rm: cannot remove 'commonsense_170k.json': No such file or directory
--2026-01-15 23:07:47--  https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/ft-training_set/commonsense_170k.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 96529722 (92M) [text/plain]
Saving to: ‘commonsense_170k.json’

commonsense_170k.js 100%[===================>]  92.06M   248MB/s    in 0.4s    

2026-01-15 23:07:48 (248 MB/s) - ‘commonsense_170k.json’ saved [96529722/96529722]

Dataset downloaded successfully!


### Evaluation dataset

In [ ]:
base_dir = "/content/DoRA/commonsense_reasoning/dataset"
os.makedirs(base_dir, exist_ok=True)

tasks = {
    "boolq": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/boolq/test.json",
    "piqa": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/piqa/test.json",
    "social_iqa": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/social_iqa/test.json",
    "hellaswag": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/hellaswag/test.json",
    "winogrande": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/winogrande/test.json",
    "ARC-Easy": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/ARC-Easy/test.json",
    "ARC-Challenge": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/ARC-Challenge/test.json",
    "openbookqa": "https://raw.githubusercontent.com/AGI-Edgerunners/LLM-Adapters/main/dataset/openbookqa/test.json"
}

for task, url in tasks.items():
    task_dir = os.path.join(base_dir, task)
    os.makedirs(task_dir, exist_ok=True)
    output_file = os.path.join(task_dir, "test.json")

    print(f"Downloading {task}...")
    !wget -q {url} -O {output_file}

###Hyperparameters
Llama -7B \
rank r: 16 (to save memory and compute) \
alpha: 64 \
dropout: 0.05 \
optimizer: adam \
lr: 2e-4, 1e-4 \
lr scheduler: linear \
batch: 16 \
warmup: 100 \
epochs: 3 \
wahere: Q, K, V, Up, Down

In [ ]:
%cd /content/DoRA/commonsense_reasoning
!chmod +x *.sh


/content/DoRA/commonsense_reasoning


###Finetuning

In [ ]:
# LLama 7B, using micro batch size 8 to save memory (out of bounds on A100)
!python finetune.py \
    --base_model 'yahma/llama-7b-hf' \
    --data_path 'commonsense_170k.json' \
    --output_dir './finetuned_result/dora_r16' \
    --batch_size 16 --micro_batch_size 8 --num_epochs 3 \
    --learning_rate 2e-4 --cutoff_len 256 --val_set_size 120 \
    --eval_step 80 --save_step 80 --adapter_name dora \
    --target_modules '["q_proj", "k_proj", "v_proj", "up_proj", "down_proj"]' \
    --lora_r 16 --lora_alpha 64 --use_gradient_checkpointing

/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Finetuning model with params:
base_model: yahma/llama-7b-hf
data_path: commonsense_170k.json
output_dir: ./finetuned_result/dora_r16
batch_size: 16
micro_batch_size: 8
num_epochs: 3
learning_rate: 0.0002
cutoff_len: 256
val_set_size: 120
use_gradient_checkpointing: True
lora_r: 16
lora_alpha: 64
lora_dropout: 0.05
lora_target_modules: None
Wdecompose_target_modules: None
dora_simple: Truebottleneck_size: 256
non_linearity: tanh
adapter_dropout: 0.0
use_parallel_adapter: False
use_adapterp: Fal

In [ ]:
!nvidia-smi


Thu Jan 15 23:04:38 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             45W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

###Evaluation

In [ ]:
!bash llama_7B_Dora_eval.sh ./finetuned_result/llama_dora_r16 0

tee: ./finetuned_result/llama_dora_r32/boolq.txt: No such file or directory
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Traceback (most recent call last):
  File "/content/DoRA/commonsense_reasoning/commonsense_evaluate.py", line 20, in <module>
    from peft import PeftModel
  File "/content/DoRA/commonsense_reasoning/peft/src/peft/__init__.py", line 30, in <module>
    from .mapping import MODEL_TYPE_TO_PEFT_MODEL_MAPPING, PEFT_TYPE_TO_CONFIG_MAPPING, get_peft_config, get_peft_model
  File "/content/DoRA/commonsense_reasoning/peft/src

### Fine tune and eval on LLama2 and LLama3 7B

In [ ]:
%cd /content/DoRA/commonsense_reasoning
# llama2 7B rank 16 alpha 64
!bash llama2_7B_DoRA.sh 16 64 ./finetuned_result/llama_2_r16_lr2e-4 0

# llama3 7B rank 16 alpha 64
!bash llama3_8B_DoRA.sh 16 64 ./finetuned_result/llama_3_r16_lr1e-4 0

/content/DoRA/commonsense_reasoning
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.12/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Traceback (most recent call last):
  File "/content/DoRA/commonsense_reasoning/finetune.py", line 25, in <module>
    from peft import (  # noqa: E402
  File "/content/DoRA/commonsense_reasoning/peft/src/peft/__init__.py", line 30, in <module>
    from .mapping import MODEL_TYPE_TO_PEFT_MODEL_MAPPING, PEFT_TYPE_TO_CONFIG_MAPPING, get_peft_config, get_peft_model
  File "/content/DoRA/commonsense_reasoning/peft/src/peft/mapping.py", line 25, in <module>
    fr